In [16]:
import numpy as np
import pandas as pd
import gzip
import io
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Yao\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Yao\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Yao\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Yao\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
df = pd.read_csv('dataset-test.csv')

In [5]:
#use a setset for performance purpose
df = df[df['subreddit'] == 'leagueoflegends']

In [6]:
pd.options.display.max_colwidth = 90
df.head()

,Unnamed: 0,author,body,normalizedBody,subreddit,subreddit_id,id,content,summary
78,1311804,Epig0n,I understand everyone wants to own every skin in the game for their favorite champions...,I understand everyone wants to own every skin in the game for their favorite champions...,leagueoflegends,t5_2rfxx,c98ysis,I understand everyone wants to own every skin in the game for their favorite champions...,"I own several legacy skins, I would die for more, but not if everyone else gets to buy..."
131,3731807,HeavenlyFace,I got some technical issues with my computer who crashed several times lately and rand...,I got some technical issues with my computer who crashed several times lately and rand...,leagueoflegends,t5_2rfxx,t3_2g161e,I got some technical issues with my computer who crashed several times lately and rand...,I think the Riot suggested link for technical issues is unsecure. \n Edit : I have no...
144,3158563,schmanthony,"Firstly, I want to reiterate what has been stated multiple times in light of this news...","Firstly, I want to reiterate what has been stated multiple times in light of this news...",leagueoflegends,t5_2rfxx,t3_23sxcz,"Firstly, I want to reiterate what has been stated multiple times in light of this news...","When commenting, don't be a dick; e-Sports is like Sports aside from several key diffe..."
170,2926104,Bc--Chronic,ARAU - All Random All Urf\n\nYou log onto league after updating a patch. When you get...,ARAU - All Random All Urf \n You log onto league after updating a patch. When you get ...,leagueoflegends,t5_2rfxx,t3_4f32f0,ARAU - All Random All Urf \n You log onto league after updating a patch. When you get ...,"Have a random game mode that randomly assigns a game mode(normal game, doom bots, hexa..."
219,1536300,SquirrellyOtter,RE your second argument: the modified smites (and their items) could be considered fle...,RE your second argument: the modified smites (and their items) could be considered fle...,leagueoflegends,t5_2rfxx,cmnuc6x,RE your second argument: the modified smites (and their items) could be considered fle...,I think smite on the ADC is at least somewhat more viable than you give it credit for.


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10748 entries, 78 to 380981
Data columns (total 9 columns):
Unnamed: 0        10748 non-null int64
author            10748 non-null object
body              10748 non-null object
normalizedBody    10748 non-null object
subreddit         10748 non-null object
subreddit_id      10748 non-null object
id                10748 non-null object
content           10748 non-null object
summary           10748 non-null object
dtypes: int64(1), object(8)
memory usage: 839.7+ KB


In [8]:
df['subreddit'].value_counts()

leagueoflegends    10748
Name: subreddit, dtype: int64

In [9]:
df.astype(str).apply(lambda s: s.str.len()).max() #max length

Unnamed: 0            7
author               20
body              16714
normalizedBody    16480
subreddit            15
subreddit_id          8
id                    9
content           16154
summary            3404
dtype: int64

In [10]:
df['summary'].str.len().agg(['mean','max','std'])

mean     140.805545
max     3404.000000
std      161.801909
Name: summary, dtype: float64

In [11]:
df['content'].str.len().agg(['mean','max','std'])

mean     1157.319408
max     16154.000000
std      1172.495042
Name: content, dtype: float64

**Pre-processing**

In [4]:
#function to pre process the text data
#use lemm, but can also try different combination of stop words,stem vs lemm, remove words
def pre_processing(data):
    stop_words = set(stopwords.words('english')) #use stop words
    #data = re.sub('(\d+)', '', data) #remove digits
    data = re.sub('\W+',' ', data) #remove special chaacters
    word_tokens = word_tokenize(data.lower()) #tokenize the string
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words] #remove stop words
    #porter_stemmer = PorterStemmer() #stem the words 
    #stemmed_words = [porter_stemmer.stem(word) for word in filtered_sentence]
    wordnet_lemmatizer = WordNetLemmatizer() #lemm the word, does better than stem the word
    lemm_words = [wordnet_lemmatizer.lemmatize(word) for word in filtered_sentence]
    return ' '.join(lemm_words)

In [23]:
data_text = df['content']
data_sum = df['summary']

In [ ]:
#bag of words in case we need to use it
#vectorizer_proc = CountVectorizer(preprocessor = pre_processing)
#data_text_proc = vectorizer_proc.fit_transform(data_text)

In [22]:
print(data_text[:5]) #show original text
data_text_proc = list(map(pre_processing, data_text)) # pre process the entire data set
print('-----------------------------------------------'*3)
print(data_text_proc[:5]) #show pre processed test

['I understand everyone wants to own every skin in the game for their favorite champions. Having started LoL at the start of S1, I quickly started to main Anivia and I am, to this day, lacking the Team Spirit skin for her. \nI do however own several other Limited Edition skins from Harrowing and other events. And I say this is not a compromise in any sense. \nLimited means limited. If Riot puts them up for sale again they lose all value and importance. They are so amazing primarily because they are not available to 90%~ of people playing the game today. \nWould you really be so in awe of Kitty Kat Katarina if you saw every single Kat player using that skin? I think not. \nRiot needs to stop trying to please all the new players begging for legacy content and start to honor the old players that have been investing in the game from the very beginning.', "I got some technical issues with my computer who crashed several times lately and randomly, and I ended being leave busted. \n Problem i

In [24]:
print(data_sum[:5]) #show original text
data_sum_proc = list(map(pre_processing, data_sum)) # pre process the entire data set
print('-----------------------------------------------'*3)
print(data_sum_proc[:5]) #show pre processed test

78     I own several legacy skins, I would die for more, but not if everyone else gets to buy...
131    I think the Riot suggested link for technical issues is unsecure. \n Edit  : I have no...
144    When commenting, don't be a dick; e-Sports is like Sports aside from several key diffe...
170    Have a random game mode that randomly assigns a game mode(normal game, doom bots, hexa...
219       I think smite on the ADC is at least somewhat more viable than you give it credit for.
Name: summary, dtype: object
---------------------------------------------------------------------------------------------------------------------------------------------
['several legacy skin would die everyone else get buy keep unique keep limited', 'think riot suggested link technical issue unsecure edit problem accessing ssl site instance access google facebook without problem nothing ssl certificate think', 'commenting dick e sport like sport aside several key difference let player step retire dignity bas

In [26]:
print(len(data_text_proc))
print(len(data_sum_proc))

10748
10748
